In [1]:
# Import necessary libraries
from Bio import SeqIO
import os
import numpy as np
import pandas as pd
from collections import defaultdict

# --- 1. Define File Paths and Parameters ---
# Input file from the 'raw' data folder
raw_file_path = os.path.join('..', 'data', '01_raw', 'SRR26872904_1.fastq')

# Output file for the processed data
processed_file_path = os.path.join('..', 'data', '02_processed', 'kmer_features.csv')

# Parameters for our pre-processing
QUALITY_THRESHOLD = 20  # Minimum average quality score to keep a sequence
K = 4  # The length of the "k-mer" to use for feature extraction

# --- 2. Data Cleaning: Quality Filtering ---
# We will read the raw FASTQ file and keep only the sequences that meet our quality criteria.
print(f"Starting quality filtering with threshold: {QUALITY_THRESHOLD}...")

high_quality_sequences = []
with open(raw_file_path, "r") as handle:
    for record in SeqIO.parse(handle, "fastq"):
        # Calculate the average quality score for the sequence
        avg_quality = np.mean(record.letter_annotations["phred_quality"])
        
        # If the average quality is above our threshold, we keep the sequence
        if avg_quality >= QUALITY_THRESHOLD:
            high_quality_sequences.append(record.seq)

print(f"Finished quality filtering.")
print(f"Original number of sequences: 1,513,460") # From previous notebook
print(f"Number of high-quality sequences: {len(high_quality_sequences)}")


# --- 3. Data Transformation: K-mer Counting (Feature Extraction) ---
# Now, we convert the clean DNA sequences (e.g., 'AGCT') into numerical vectors.
# We do this by counting the occurrences of all possible DNA "words" of length K.
print(f"\nStarting k-mer counting with k={K}...")

def get_kmers(sequence, k):
    """Function to break a sequence into k-mers."""
    kmers = []
    for i in range(len(sequence) - k + 1):
        kmers.append(str(sequence[i:i+k]))
    return kmers

# A list to store the k-mer counts for each sequence
kmer_counts_list = []

# Loop through all the high-quality sequences
for seq in high_quality_sequences:
    # Create a dictionary to store counts for the current sequence
    # defaultdict is useful here because it provides a default value for a key that doesn't exist
    counts = defaultdict(int)
    
    # Get all k-mers from the sequence
    kmers = get_kmers(seq, K)
    
    # Count the frequency of each k-mer
    for kmer in kmers:
        counts[kmer] += 1
    
    kmer_counts_list.append(counts)

print("Finished k-mer counting.")

# --- 4. Create and Save the Feature Matrix ---
# Convert the list of dictionaries into a pandas DataFrame.
# This DataFrame is our final feature matrix, ready for machine learning.
print("\nCreating feature matrix (DataFrame)...")

feature_df = pd.DataFrame(kmer_counts_list)

# Replace any missing values (NaN) with 0. This happens when a k-mer
# doesn't appear in a particular sequence.
feature_df = feature_df.fillna(0)

# Save the DataFrame to a CSV file in the 'processed' data folder
feature_df.to_csv(processed_file_path, index=False)

print(f"Successfully created and saved the feature matrix.")
print(f"Shape of the final DataFrame: {feature_df.shape}")
print(f"File saved to: {processed_file_path}")

# Display the first few rows of our final dataset
print("\n--- First 5 rows of the processed data ---")
print(feature_df.head())


Starting quality filtering with threshold: 20...
Finished quality filtering.
Original number of sequences: 1,513,460
Number of high-quality sequences: 1509422

Starting k-mer counting with k=4...
Finished k-mer counting.

Creating feature matrix (DataFrame)...
Successfully created and saved the feature matrix.
Shape of the final DataFrame: (1509422, 504)
File saved to: ..\data\02_processed\kmer_features.csv

--- First 5 rows of the processed data ---
   CNGT  NGTC  GTCA  TCAC  CACG  ACGT  CGTT  GTTC  TTCG  TCGT  ...  ANGT  \
0   1.0   1.0   1.0   2.0   2.0   1.0   2.0   1.0   1.0   1.0  ...   0.0   
1   0.0   0.0   0.0   1.0   2.0   1.0   1.0   2.0   1.0   0.0  ...   0.0   
2   1.0   1.0   1.0   4.0   4.0   1.0   2.0   1.0   1.0   1.0  ...   0.0   
3   0.0   0.0   0.0   1.0   1.0   0.0   2.0   0.0   1.0   0.0  ...   0.0   
4   0.0   0.0   0.0   1.0   2.0   1.0   1.0   2.0   1.0   0.0  ...   0.0   

   TTTN  ATNA  ACNT  ANTG  ANCC  AANT  TNCN  NCNA  GTCN  
0   0.0   0.0   0.0   0.0   0.